# Top 1 million websites

Alexa Top 1m Site as a file (may be old):

http://s3.amazonaws.com/alexa-static/top-1m.csv.zip

Magic Million - Top Million Sites Update Daily

https://blog.majestic.com/development/majestic-million-csv-daily/


In [ ]:
import pandas as pd
magic_top1m_df = pd.read_csv('/Datasets/Top1m/top1m-websites-2020-09-majestic_million.csv')

In [ ]:
magic_top1m_df

In [ ]:
alexa_top1m_df = pd.read_csv('/Datasets/Top1m/amazon-s3-alexa-top-1m.csv')

In [ ]:
alexa_top1m_df

# Alexa Web Information Service

API Key: ** Key to use **

Developer Portal: https://awis.alexa.com/

Example JSON request:

https://awis.api.alexa.com/api?Action=UrlInfo&Count=10&ResponseGroup=Rank,RankByCountry,AdultContent,Language,LinksInCount,SiteData&Start=1&Url=cnn.com&Output=json

With x-api-key header


In [2]:
# Python API request
import json
import requests
import tldextract
import urllib
import socket
from bs4 import BeautifulSoup

def get_awis_website(url):
    
    api_key = '****'
    api_url_base = 'https://awis.api.alexa.com/api?Action=UrlInfo&Start=1&Count=10&Output=json'
    
    response_group = 'Rank,RankByCountry,AdultContent,Language,LinksInCount,SiteData'
    headers = {'x-api-key':api_key}

    api_url = api_url_base + '&ResponseGroup=' + response_group + '&Url=' + url
    response = requests.get(api_url, headers=headers)
    if response.status_code == 200:
        return json.loads(response.content.decode('utf-8'))
    else:
        return None
    
def get_awis_batch_websites(url_list):
    # Up to 5 allowed in one call
    
    api_key = '****'
    api_url_base = 'https://awis.api.alexa.com/api?Action=UrlInfo&Output=json'
    
    response_group = 'Rank,RankByCountry,AdultContent,Language,LinksInCount,SiteData'
    headers = {'x-api-key':api_key}
    
    url_list_parameter = ''
    for idx, url in enumerate(url_list):
        url_list_parameter += '&UrlInfo.'+str(idx + 1)+'.Url='+url

    api_url = api_url_base + '&UrlInfo.Shared.ResponseGroup=' + response_group + url_list_parameter
    response = requests.get(api_url, headers=headers)
    if response.status_code == 200:
        return json.loads(response.content.decode('utf-8'))
    else:
        return None

def hostname_resolves(hostname):
    try:
        socket.gethostbyname(hostname)
        return 1
    except socket.error:
        return 0

def get_website_data(url):
    USER_AGENT = "Mozilla/5.0 (Linux; Android 7.0; SM-G930V Build/NRD90M) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.125 Mobile Safari/537.36"
    headers = {"user-agent" : USER_AGENT}
    resp = requests.get('http://' + url, headers=headers)
    soup = BeautifulSoup(resp.content, "html.parser")
    print(soup.title)
    website_data = {
        'title': soup.title.string.strip('\n'),
        'description': soup.description.string.strip('\n'),
        'keywords': soup.keywords.string.strip('\n'),
        'image': '',
        'favicon': ''
    }
    return { 'title': soup.title.string.strip('\n'), 'description': soup.description.string.strip('\n') }
    
    

In [260]:
url = 'cnn.com'
USER_AGENT = "Mozilla/5.0 (Linux; Android 7.0; SM-G930V Build/NRD90M) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.125 Mobile Safari/537.36"
headers = {"user-agent" : USER_AGENT}
resp = requests.get('http://' + url, headers=headers)
soup = BeautifulSoup(resp.content, "html.parser")
print(soup.title.string)
print(soup.title.description)

CNN - Breaking News, Latest News and Videos
None


In [ ]:
description = soup.find("meta",  property="og:description")

print(description["content"] if description else "No meta description given")
soup.find('meta',attrs={'name':'keywords'})['content']
soup.find('meta',attrs={'name':'description'})['content']
soup.find("meta",  property="og:description")['content']

In [264]:
description = soup.find("meta",  property="og:description")
description

<meta content="View the latest news and breaking news today for U.S., world, weather, entertainment, politics and health at CNN.com." property="og:description"/>

In [255]:
soup.find("meta",  property="og:type")

<meta content="website" property="og:type"/>

In [ ]:
# Single lookup
url_to_get = 'cnn.com'
awis_details_single = get_awis_website(url_to_get)['Awis']['Results']['Result']['Alexa']
awis_details_single

In [ ]:
# Batch results
results_list = get_awis_batch_websites(['cnn.com','yahoo.com','instagram.com'])['Awis']['Results']['Result']

Items to get from direct lookup:

* image
* favicon
* title
* description
* keywords

og:tags are here: https://developers.facebook.com/docs/sharing/webmasters/


In [257]:
get_website_title('davidyurman.com/')

<title>David Yurman | Designer Jewelry &amp; Watches for Women and Men </title>


'David Yurman | Designer Jewelry & Watches for Women and Men '

In [256]:
url_batch = ['cnn.com','yahoo.com','instagram.com','facebook.com','davidyurman.com']
urls_to_check = []

for url in url_batch:
    print(url)
    if hostname_resolves(url):
        urls_to_check.append(url)
    
print(urls_to_check)        
results_list = get_awis_batch_websites(urls_to_check)['Awis']['Results']['Result']

for idx, awis_details in enumerate(results_list):
    url_requested = awis_details['Alexa']['Request']['Arguments']['Argument'][0]['Value']
    data_url = awis_details['Alexa']['ContentData']['DataUrl']
    domain = tldextract.extract(data_url).domain
    subdomain = tldextract.extract(data_url).subdomain
    suffix = tldextract.extract(data_url).suffix
    title = awis_details['Alexa']['ContentData']['SiteData']['Title']
    if title == data_url:
        title = get_website_title(data_url)
    if title == '':
        title = domain
    description = awis_details['Alexa']['ContentData']['SiteData']['Description']
    adult_content = awis_details['Alexa']['ContentData']['AdultContent']
    language_code = awis_details['Alexa']['ContentData']['Language']
    if language_code:
        language = language_code['Locale']
    else:
        language = ''
    links_in_count = awis_details['Alexa']['ContentData']['LinksInCount']
    rank = awis_details['Alexa']['TrafficData']['Rank']
    rank_by_country = awis_details['Alexa']['TrafficData']['RankByCountry']['Country']
    for country_rank in rank_by_country:
        if country_rank['@Code'] == 'US':
            rank_usa = country_rank['Rank']
    print(title + '- rank: ' + rank)

cnn.com
yahoo.com
instagram.com
facebook.com
davidyurman.com
['cnn.com', 'yahoo.com', 'instagram.com', 'facebook.com', 'davidyurman.com']
CNN - Cable News Network- rank: 79
Yahoo!- rank: 10
<title>
Instagram
</title>


KeyError: 'Description'

# Metadata parsing


In [35]:
import lxml
import metadata_parser
USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:65.0) Gecko/20100101 Firefox/65.0"
parser_headers = {'User-Agent': USER_AGENT}

In [20]:
page = metadata_parser.MetadataParser(url="https://www.britannica.com/place/Bora-Bora")

In [21]:
print( page.metadata)

{'og': {'type': 'ARTICLE', 'title': 'Bora-Bora | Facts, History, & Points of Interest', 'description': 'Bora-Bora, volcanic island, Iles Sous le Vent (Leeward Islands), in the Society Islands of French Polynesia. It lies in the central South Pacific Ocean, about 165 miles (265 km) northwest of Tahiti. It is surrounded by coral reefs. Vaitape, the principal village and administrative center, is on the west coast.', 'site_name': 'Encyclopedia Britannica', 'url': 'https://www.britannica.com/place/Bora-Bora', 'image': 'https://cdn.britannica.com/57/99557-050-213502D2/Hut-dwellings-Bora-Bora-Society-Islands-French-Polynesia.jpg', 'image:type': 'image/jpeg'}, 'meta': {'charset': 'utf-8', 'Content-Type': 'text/html; charset=UTF-8', 'viewport': 'width=device-width, initial-scale=1.0', 'p:domain_verify': '5247c07bac63b6b3250f68cfe355c9a2', 'fb:pages': '74442380906', 'description': 'Bora-Bora, volcanic island, Iles Sous le Vent (Leeward Islands), in the Society Islands of French Polynesia. It li

In [10]:
print(page.get_metadatas('description'))

['Bora-Bora, volcanic island, Iles Sous le Vent (Leeward Islands), in the Society Islands of French Polynesia. It lies in the central South Pacific Ocean, about 165 miles (265 km) northwest of Tahiti. It is surrounded by coral reefs. Vaitape, the principal village and administrative center, is on the west coast.']


In [ ]:
page = metadata_parser.MetadataParser(url="https://money.usnews.com", url_headers=parser_headers, requests_timeout=10)

In [36]:
print(page.get_metadatas('description'))


None


In [ ]:
page = metadata_parser.MetadataParser(url="https://www.mayoclinic.org/diseases-conditions/dry-skin/expert-answers/cracked-thumb-tip/faq-20450998")

In [29]:
print(page.get_metadatas('image'))

None


In [31]:
page.get_metadata_link('image', allow_encoded_uri=True)

'https://i.ytimg.com/vi/_WS9w10ygpU/maxresdefault.jpg'

In [9]:
import metadata_parser

In [32]:
page = metadata_parser.MetadataParser(url="https://www.youtube.com/watch?v=_WS9w10ygpU")


In [33]:
import requests

In [36]:
page_header = requests.head("https://www.bloomberg.com/news/articles/2020-11-07/after-crushing-covid-melbourne-emerges-wary-and-more-sedate", headers=parser_headers, allow_redirects=True)

In [37]:
page_header.headers

{'Connection': 'keep-alive', 'Content-Length': '53335', 'Content-Type': 'text/html; charset=utf-8', 'Server': 'openresty', 'Content-Security-Policy': "frame-ancestors 'self' http://*.bloomberg.com https://*.bloomberg.com, upgrade-insecure-requests", 'ETag': 'W/"507e1-ZBrDePFxRoPo5ZWABdyrcu6EhE4"', 'Content-Encoding': 'gzip', 'Cache-Control': 'public, max-age=120', 'Fastly-Restarts': '1', 'Accept-Ranges': 'bytes', 'Date': 'Mon, 09 Nov 2020 07:22:58 GMT', 'Age': '0', 'Set-Cookie': '_pxhd=41ed4fa809e698a598f8b9166918921b51ea71e2530ebc527161fa6e98a9bc39:649f01d1-225c-11eb-adc2-2b70b001dcf5; Expires=Fri, 01 Jan 2021 00:00:00 GMT; path=/;', 'X-Served-By': 'cache-mia11328-MIA', 'X-Cache': 'MISS', 'X-Cache-Hits': '0', 'Strict-Transport-Security': 'max-age=31557600', 'Vary': 'Accept-Encoding, Accept-Encoding'}

In [54]:
answer_link = "https://www.bloomberg.com/news/articles/2020-11-07/after-crushing-covid-melbourne-emerges-wary-and-more-sedate"
page = requests.head(answer_link, headers=parser_headers, timeout=6, allow_redirects=True)

In [49]:
header = {"user-agent": "lazybot" ,'referer':'https://www.google.com/'}
r = requests.get("http://webcache.googleusercontent.com/search?strip=1&q=cache:www.bloomberg.com/news/articles/2020-11-07/after-crushing-covid-melbourne-emerges-wary-and-more-sedate",headers=header)

In [57]:
answer_link_google_cache = f"http://webcache.googleusercontent.com/search?q=cache:{answer_link.partition('://')[2]}"

In [58]:
answer_link_google_cache

'http://webcache.googleusercontent.com/search?q=cache:www.bloomberg.com/news/articles/2020-11-07/after-crushing-covid-melbourne-emerges-wary-and-more-sedate'

In [50]:
r.content

b'<meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><base href="https://www.bloomberg.com/news/articles/2020-11-07/after-crushing-covid-melbourne-emerges-wary-and-more-sedate"><style>body{margin-left:0;margin-right:0;margin-top:0}#bN015htcoyT__google-cache-hdr{background:#f8f9fa;font:13px arial,sans-serif;text-align:left;color:#202124;border:0;margin:0;border-bottom:1px solid #dadce0;line-height:16px;padding:16px 28px 24px 28px}#bN015htcoyT__google-cache-hdr *{display:inline;font:inherit;text-align:inherit;color:inherit;line-height:inherit;background:none;border:0;margin:0;padding:0;letter-spacing:0}#bN015htcoyT__google-cache-hdr a{text-decoration:none;color:#4b11a8}#bN015htcoyT__google-cache-hdr a:hover{text-decoration:underline}#bN015htcoyT__google-cache-hdr a:visited{color:#4b11a8}#bN015htcoyT__google-cache-hdr div{display:block;margin-top:4px}#bN015htcoyT__google-cache-hdr b{font-weight:bold;display:inline-block;direction:ltr}</style><div id="bN015htcoyT__google-ca

In [37]:
import wikipedia

In [50]:
mystr = wikipedia.page("Maldives").summary

In [51]:
mystr.partition('\n')[0]

'The Maldives (, US:  (listen); Dhivehi: ދިވެހިރާއްޖެ Dhivehi Raajje), officially the Republic of Maldives, is a small island nation in South Asia, situated in the Arabian Sea of the Indian Ocean. It lies southwest of Sri Lanka and India, about 700 kilometres (430 mi) from the Asian continent\'s mainland. The chain of 26 atolls stretches from Ihavandhippolhu Atoll in the north to Addu Atoll in the south to the equator. Comprising a territory spanning roughly 298 square kilometres (115 sq mi), the Maldives is one of the world\'s most geographically dispersed sovereign states as well as the smallest Asian country by land area and population, with around 515,696 inhabitants. Malé is the capital and the most populated city, traditionally called the "King\'s Island" where the ancient royal dynasties ruled for its central location.The Maldives archipelago is located on the Chagos-Laccadive Ridge, a vast submarine mountain range in the Indian Ocean; this also forms a terrestrial ecoregion, to

In [47]:
title = 'Panama Canal - Wikipedia'
page_title = title.partition(' - Wikipedia')[0]

In [48]:
page_title

'Panama Canal'

In [57]:
myurl = "https://en.wikipedia.org/wiki/Donald_Trump"
right_slash = myurl.rfind('/')
page_to_get = myurl[right_slash + 1:].replace('_', ' ')
page_to_get

'Donald Trump'

In [56]:
myurl[29:]


'/Donald_Trump'

In [8]:
import favicon
import tldextract

In [3]:
favicon.get("https://www.cnn.com/2020/10/16/politics/trump-tax-returns-nbc-town-hall-savannah-guthrie/index.html", timeout=2)

[Icon(url='https://www.cnn.com/favicon.ie9.ico', width=0, height=0, format='ico'),
 Icon(url='https://cdn.cnn.com/cnn/.e/img/3.0/global/misc/apple-touch-icon.png', width=0, height=0, format='png'),
 Icon(url='https://cdn.cnn.com/cnnnext/dam/assets/201015223113-14-deuling-town-halls-super-tease.jpg', width=0, height=0, format='jpg')]

In [4]:
favicon.get("https://www.britannica.com/topic/Perseus-Greek-mythology", timeout=2)


[Icon(url='https://cdn.britannica.com/45/3445-004-07CCF23B/Perseus-bronze-sculpture-Benvenuto-Cellini-Loggia-dei.jpg', width=0, height=0, format='jpg')]

In [12]:
answer_link = "https://www.britannica.com/topic/Perseus-Greek-mythology"
answer_website = 'http://'+tldextract.extract(answer_link).subdomain + "." + tldextract.extract(answer_link).domain + '.' + tldextract.extract(answer_link).suffix
icons_list = favicon.get(answer_website, timeout=2)
source_favicon = icons_list[0]

In [13]:
answer_website

'http://www.britannica.com'

In [17]:
icons_list[0].url

'https://www.britannica.com:443/favicon.ico'

In [19]:
for icon in icons_list:
    if 'png' in icon.format or 'jpg' in icon.format:
        source_favicon = icon.url
        break
    else:
        source_favicon = icon.url
        

In [20]:
source_favicon

'https://www.britannica.com/resources/images/EBLogo.jpg'

In [27]:
import requests
from bs4 import BeautifulSoup
import fake_useragent
# desktop user-agent
DEFAULT_USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:65.0) Gecko/20100101 Firefox/65.0"
# mobile user-agent
MOBILE_USER_AGENT = "Mozilla/5.0 (Linux; Android 7.0; SM-G930V Build/NRD90M) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.125 Mobile Safari/537.36"
# iphone user agent
IPHONE_USER_AGENT = "Mozilla/5.0 (iPhone; CPU iPhone OS 6_1 like Mac OS X) AppleWebKit/536.26 (KHTML; like Gecko) Mobile/10B142 iPhone4;1 BingWeb/3.03.1428.20120423"
# bot agent
BOT_USER_AGENT = "Mozilla/5.0 (compatible; Lazybot/2.1; +http://www.lazyweb.ai/bot)"
GOOGLE_BOT_AGENT = "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)"
# Google bot agent
HEADERS_BASE = {"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8", "accept-encoding": "gzip, deflate, sdch, br", "accept-language": "en-US,en;q=0.8,ms;q=0.6", "upgrade-insecure-requests": "1", "referrer": "https://www.google.com", 'Access-Control-Allow-Origin': '*','Access-Control-Allow-Methods': 'GET','Access-Control-Allow-Headers': 'Content-Type','Access-Control-Max-Age': '3600'}

PROXY_HEADERS_BASE = {"x-lazyweb-origin": "lazyweb.ai","accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8", "accept-encoding": "gzip, deflate, sdch, br", "accept-language": "en-US,en;q=0.8,ms;q=0.6", "upgrade-insecure-requests": "1", "referrer": "https://www.google.com", 'Access-Control-Allow-Origin': '*','Access-Control-Allow-Methods': 'GET','Access-Control-Allow-Headers': 'Content-Type','Access-Control-Max-Age': '3600'}
BOT_HEADERS_BASE = {"x-lazyweb-origin": "lazyweb.ai"}
# User agent db download
UA_DB = "https://fake-useragent.herokuapp.com/browsers/0.1.11"



In [30]:
ua_download = requests.get(UA_DB, allow_redirects=True)
ua_location = 'fake_useragent-%s.json' % fake_useragent.VERSION
open(ua_location, 'wb').write(ua_download.content)
ua = fake_useragent.UserAgent(path=ua_location, fallback=DEFAULT_USER_AGENT)
user_agent = ua.random
bot_headers = {'user-agent': GOOGLE_BOT_AGENT}
bot_headers.update(BOT_HEADERS_BASE)
parser_headers = {'user-agent': ua.random}
proxy_headers = {"user-agent": ua.random}
proxy_headers.update(PROXY_HEADERS_BASE)
local_headers = {"user-agent": ua.random}
local_headers.update(HEADERS_BASE)

# Page Content Workings

In [31]:
local_headers

{'user-agent': 'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.14 (KHTML, like Gecko) Chrome/24.0.1292.0 Safari/537.14',
 'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
 'accept-encoding': 'gzip, deflate, sdch, br',
 'accept-language': 'en-US,en;q=0.8,ms;q=0.6',
 'upgrade-insecure-requests': '1',
 'referrer': 'https://www.google.com',
 'Access-Control-Allow-Origin': '*',
 'Access-Control-Allow-Methods': 'GET',
 'Access-Control-Allow-Headers': 'Content-Type',
 'Access-Control-Max-Age': '3600'}

In [24]:
url = "https://au.linkedin.com/in/stevevamos"
link_proxy_url = "https://lazyweb-proxy.herokuapp.com/https://au.linkedin.com/in/stevevamos"


In [32]:
r = requests.get(url, headers=local_headers, timeout=15)
html = BeautifulSoup(r.content, 'html.parser')

In [21]:
html.title

In [33]:
r

<Response [999]>

In [5]:
response = {"sessionAttributes":{"currentSearchResult":"{\"results_type\":\"Snippet\",\"answer\":\"A growth hacker is someone who uses creative, low-cost strategies to help businesses acquire and retain customers. Sometimes growth hackers are also called growth marketers, but growth hackers are not simply marketers.\",\"type\":\"article\",\"title\":\"What Is Growth Hacking? A Simple Definition + How to Get Started\",\"description\":\"What is growth hacking? It's not just a meaningless buzzword. Read this simple growth hacking definition to find out how to grow your business FAST.\",\"image\":\"https://cdn.optinmonster.com/wp-content/uploads/2014/10/fbdefaultimage.png\",\"link\":\"https://optinmonster.com/glossary/growth-hacking/\",\"source\":\"OptinMonster\",\"favicon\":\"\",\"results\":[{\"title\":\"What Is Growth Hacking? A Simple Definition + How to Get ...\",\"link\":\"https://optinmonster.com/glossary/growth-hacking/\",\"desc\":\"\"},{\"title\":\"Definitive Guide to Growth Hacking - QuickSprout\",\"link\":\"https://www.quicksprout.com/growth-hacking/\",\"desc\":\"\"},{\"title\":\"Growth Hacking Made Simple: A Step-by-Step Guide - Neil Patel\",\"link\":\"https://neilpatel.com/what-is-growth-hacking/\",\"desc\":\"\"},{\"title\":\"Growth hacking - Wikipedia\",\"link\":\"https://en.wikipedia.org/wiki/Growth_hacking\",\"desc\":\"\"},{\"title\":\"23 Proven Growth Hacking Examples You Can Steal to Gain ...\",\"link\":\"https://optinmonster.com/growth-hacking-examples/\",\"desc\":\"\"},{\"title\":\"Growth Hacking Explained: What Is It and How Can It Benefit ...\",\"link\":\"https://digitalmarketinginstitute.com/blog/growth-hacking-explained-can-benefit\",\"desc\":\"\"},{\"title\":\"GrowthHackers - Premier Growth Community - GrowthHackers\",\"link\":\"https://growthhackers.com/posts\",\"desc\":\"\"},{\"title\":\"What is Growth Hacking? What does a Growth Hacker do + ...\",\"link\":\"https://growwithward.com/what-is-growth-hacking/\",\"desc\":\"\"},{\"title\":\"Growth Hacking - Optimizely\",\"link\":\"https://www.optimizely.com/optimization-glossary/growth-hacking/\",\"desc\":\"\"},{\"title\":\"Growth Hacking 101: What It Is and How to Do It | WordStream\",\"link\":\"https://www.wordstream.com/blog/ws/2015/06/02/growth-hacking\",\"desc\":\"\"},{\"title\":\"Hacking Growth: How Today's Fastest-Growing Companies ...\",\"link\":\"https://www.amazon.com/Hacking-Growth-Fastest-Growing-Companies-Breakout/dp/045149721X\",\"desc\":\"\"},{\"title\":\"Growth Hacking: Common Myths and a Process that Actually ...\",\"link\":\"https://cxl.com/blog/growth-hacking/\",\"desc\":\"\"},{\"title\":\"The Growth Hacking Playbook: Your Ultimate List of Growth ...\",\"link\":\"https://blog.hubspot.com/sales/growth-hacking-websites\",\"desc\":\"\"},{\"title\":\"19 Growth Hacking Examples That You Can Copy-Paste ...\",\"link\":\"https://www.grow-force.com/growth-hacking-examples/\",\"desc\":\"\"},{\"title\":\"What is growth hacking? Definition - Omniconvert\",\"link\":\"https://www.omniconvert.com/what-is/growth-hacking/\",\"desc\":\"\"},{\"title\":\"7 Growth Hacking Examples to Inspire Your Own Marketing\",\"link\":\"https://mention.com/en/blog/growth-hacking-examples/\",\"desc\":\"\"},{\"title\":\"What is Growth Hacking, Why it Is Important, Process and ...\",\"link\":\"https://kajabi.com/blog/what-is-growth-hacking-why-it-is-important-process-and-examples\",\"desc\":\"\"},{\"title\":\"Growth Marketing Guide: Advanced Tactics and Hacks\",\"link\":\"https://www.julian.com/guide/growth/intro\",\"desc\":\"\"},{\"title\":\"What Is Growth Hacking? A Definitive Guide | Built In\",\"link\":\"https://builtin.com/growth-hacking/growth-hacking-guide\",\"desc\":\"\"},{\"title\":\"Growth Hacker is the new VP Marketing - andrewchen\",\"link\":\"https://andrewchen.co/how-to-be-a-growth-hacker-an-airbnbcraigslist-case-study/\",\"desc\":\"\"}]}","searchResultHistory":"[{}]"},"dialogAction":{"type":"Close","fulfillmentState":"Fulfilled","message":{"contentType":"PlainText","content":"A growth hacker is someone who uses creative, low-cost strategies to help businesses acquire and retain customers. Sometimes growth hackers are also called growth marketers, but growth hackers are not simply marketers."}}}

In [6]:
len(str(response))

3961

# StackOverflow Top Questions

In [3]:
# Stackoverflow Top Questions
url = 'https://stackoverflow.com/questions?sort=votes'
r = requests.get(url)
html = BeautifulSoup(r.content, 'html.parser')
questions_containers = html.findAll('div', class_="question-summary")
question_list = []
for question_link in questions_containers:
    question = question_link.find('a').get_text(' ')
    question_list.append(question)
question_list

['Why is processing a sorted array faster than processing an unsorted array?',
 'How do I undo the most recent local commits in Git?',
 'How do I delete a Git branch locally and remotely?',
 "What is the difference between 'git pull' and 'git fetch'?",
 'What does the “yield” keyword do?',
 'What is the correct JSON content type?',
 "How do I undo 'git add' before commit?",
 'What is the “-->” operator in C++?',
 'How do I rename a local Git branch?',
 'How can I remove a specific item from an array?',
 'What and where are the stack and heap?',
 'Can comments be used in JSON?',
 'How do I check if an element is hidden in jQuery?',
 'Why does HTML think “chucknorris” is a color?',
 'What does “use strict” do in JavaScript, and what is the reasoning behind it?',
 'How do I redirect to another webpage?',
 'How do I force “git pull” to overwrite local files?',
 'How to modify existing, unpushed commit messages?',
 'How do JavaScript closures work?',
 'How do I revert a Git repository to a 